In [1]:
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.optimization.algorithms import MinimumEigenOptimizer, CplexOptimizer
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus

from helpers import create_models
DIR = 'TEST_DATA/'
Q_SEED = 10598 # as used in most issues
aqua_globals.random_seed = Q_SEED
shots = 4096
penalty = 1

In [2]:
# select linear program to solve
qps = create_models(DIR, penalty)
qp = qps['test_3']

In [3]:
# init classical Optimizers
optimizer = COBYLA() # SLSQP is default in Class VQE
cplex = CplexOptimizer()

In [4]:
# solve qps with Minimum Eigen Optimizer QAOA
backend = BasicAer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend,
                                   seed_simulator=Q_SEED,
                                   seed_transpiler=Q_SEED,
                                   shots=shots)  # set parameters like noise model, error mitigation and coupling map...

# QAOA parameters:

p = 2 # circuit depth grows linear with p x qp.get_num_linear_constraints()
initial_point = np.zeros(2 * p)  # default in this setting... Issue : QAOA no longer generates random initial points #1417
qaoa_mes = QAOA(p=p, quantum_instance=quantum_instance, optimizer=optimizer, initial_point=initial_point)


qaoa = MinimumEigenOptimizer(qaoa_mes, penalty=qp.penalty)

In [5]:
cplex.solve(qp)

optimal function value: -2.0
optimal value: [0. 0. 1.]
status: SUCCESS

In [6]:
print("number of qubits: ",qp.qubo.get_num_vars())

number of qubits:  8


In [7]:
res = qaoa.solve(qp)
res

optimal function value: -2.0
optimal value: [0. 0. 1.]
status: SUCCESS

In [8]:
qp.is_feasible(res.x)

True